In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
start_time=datetime.strptime("2016-07-19 00:00:00",'%Y-%m-%d %H:%M:%S')

In [2]:
df = pd.read_csv('../dataSets/training/weather_training_update.csv')
#没有第83天的数据
Weather={}
df=df[144:]
for idx in range(df.shape[0]):
    line=df.iloc[idx]
    time=str(line['date'])+' '+str(line['hour'])+':00:00'
    stime=datetime.strptime(time,'%Y-%m-%d %H:%M:%S')
    day=(stime-start_time).days
    twn_start=(stime.hour)*3
    for twn in range(twn_start,twn_start+9):
        if day not in Weather.keys():
            Weather[day]=np.zeros(72*7).reshape(72,7)
        Weather[day][twn]=line[2:]
        

In [3]:
df = pd.read_csv('../dataSets/training/training_20min_avg_travel_time.csv')
trajectories_train = {}
#trajectories_train 
#shape是91*72*15,其中15：0:avg_travel_time;[1:8]:Weather信息;[8:16]：分别是前7天对应的avg_travel_time
for idx in range(df.shape[0]):
    line = df.iloc[idx]
    route = str(line['intersection_id']+str(line['tollgate_id']))
    stime = datetime.strptime(line['time_window'][1:20],'%Y-%m-%d %H:%M:%S')
    day = (stime - start_time).days
    tw_n = (stime.hour * 60 + stime.minute)/20
    
    if route not in trajectories_train.keys():
        trajectories_train[route] = np.zeros(72*91*15).reshape(91,72,15)
    
    trajectories_train[route][day][tw_n][0] = line['avg_travel_time']
    if day!=83:
        trajectories_train[route][day][tw_n][1:8]=Weather[day][tw_n]
    if day>6:
        k=7
        for i in range(8,15):
            trajectories_train[route][day][tw_n][i]=trajectories_train[route][day-k][tw_n][0]
            k=k-1

In [4]:
weather_history = trajectories_train

## 综合

In [5]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
from copy import deepcopy

df = pd.read_csv('../dataSets/training/training_20min_avg_travel_time.csv')


trajectories_train = {}
start_time  = datetime.strptime("2016-07-19 00:00:00",'%Y-%m-%d %H:%M:%S')

for idx in range(df.shape[0]):
    line = df.iloc[idx]
    route = str(line['intersection_id']+str(line['tollgate_id']))
    stime = datetime.strptime(line['time_window'][1:20],'%Y-%m-%d %H:%M:%S')
    day = (stime - start_time).days
    tw_n = (stime.hour * 60 + stime.minute)/20
    
    if route not in trajectories_train.keys():
        trajectories_train[route] = np.zeros(72*91).reshape(91,72)
    trajectories_train[route][day][tw_n] = line['avg_travel_time']

In [6]:
def TrueFalseListCombine(TFlist1,TFlist2):
    return [l1 and l2 for l1,l2 in zip(TFlist1,TFlist2)]

In [7]:
def ExceptionOutlier(data_list):
    '''
    shape(m,n),n timewindows,m days
    根据所给数据data_list中的时间窗的平均值和标准差来去除异常
    '''
    Mean = pd.DataFrame([np.mean(i) for i in data_list])
    mean_low = Mean > Mean.quantile(0.1)
    mean_up = Mean < Mean.quantile(0.9)
    TF = TrueFalseListCombine(mean_low.values,mean_up.values)
    mean_index = Mean[TF].index.values
    
    Std = pd.DataFrame([np.std(i) for i in data_list])
    std_low = Std > Std.quantile(0.1)
    std_up = Std < Std.quantile(0.9)
    TF = TrueFalseListCombine(std_low.values,std_up.values)
    std_index = Std[TF].index.values
    
    valid_index = list(set(mean_index)&set(std_index))
    
    return valid_index # i means minues i+1 day   

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.cross_validation import train_test_split
import xgboost as xgb

##数据预处理
## shuffle
## split
data = trajectories_train
#X_train,X_test,y_train,y_test = train_test_split(data[:,17:23],data[:,24],test_size=0.2,random_state = 33)

##对每个路径的每个时间段的学习一个学习器
clfs = {};
x_tw_list = [np.arange(18,24),np.arange(19,25),np.arange(20,26),np.arange(21,27),np.arange(22,28),np.arange(23,29),\
             np.arange(45,51),np.arange(46,52),np.arange(47,53),np.arange(48,54),np.arange(49,55),np.arange(50,56)]
y_tw_list = [24,25,26,27,28,29,51,52,53,54,55,56]

for route in data.keys():
    for tw_x,tw_y in zip(x_tw_list,y_tw_list):
        #X = data[route][:,tw_x]
        y = data[route][:,tw_y]
        X = np.append(trajectories_train[route][:,tw_x],weather_history[route][:,tw_y,[1,2,3,4,5,6,7,14]],axis=1)
        idx = ExceptionOutlier(X)
        clf = xgb.XGBRegressor(max_depth=3,n_estimators=120,learning_rate=0.05).fit(X[idx],y[idx])
        if route not in clfs.keys():
            clfs[route] = [clf]
        else:
            clfs[route].append(clf)

## 加载测试数据

In [9]:
df = pd.read_csv('../dataSets/testing_phase1/weather (table 7)_test1.csv')

Weather_test={}
for idx in range(df.shape[0]):
    line=df.iloc[idx]
    time=str(line['date'])+' '+str(line['hour'])+':00:00'
    stime=datetime.strptime(time,'%Y-%m-%d %H:%M:%S')
    day=(stime-start_time).days
    twn_start=(stime.hour)*3
    for twn in range(twn_start,twn_start+9):
        if day not in Weather_test.keys():
            Weather_test[day]=np.zeros(72*7).reshape(72,7)
        Weather_test[day][twn]=line[2:] 

In [10]:
## 10.18 - 10.24号
df = pd.read_csv('../dataSets/testing_phase1/trajectories_20min_avg_travel_time.csv')
trajectories_test = {}
start_time=datetime.strptime("2016-10-18 00:00:00",'%Y-%m-%d %H:%M:%S')

for idx in range(df.shape[0]):
    line = df.iloc[idx]
    route = str(line['intersection_id']+str(line['tollgate_id']))
    stime = datetime.strptime(line['time_window'][1:20],'%Y-%m-%d %H:%M:%S')
    day = (stime - start_time).days
    tw_n = (stime.hour * 60 + stime.minute)/20
    
    if route not in trajectories_test.keys():
        trajectories_test[route] = np.zeros(72*7).reshape(7,72)
    trajectories_test[route][day][tw_n] = line['avg_travel_time']

In [11]:
#把测式数据拼接到训练数据后面，便于提取特征
for route in trajectories_train.keys():
    trajectories_train[route] = np.append(trajectories_train[route],trajectories_test[route],axis=0)

In [12]:
## 对每个路径的每段进行预测
x_tw_list = [np.arange(18,24),np.arange(19,25),np.arange(20,26),np.arange(21,27),np.arange(22,28),np.arange(23,29)\
             ,np.arange(45,51),np.arange(46,52),np.arange(47,53),np.arange(48,54),np.arange(49,55),np.arange(50,56)]
y_tw_list = [24,25,26,27,28,29,51,52,53,54,55,56]

test_data = trajectories_train
for route in test_data.keys():
    i = 0
    for tw_x,tw_y in zip(x_tw_list,y_tw_list):
        for day in range(91,98):
            test = np.append(test_data[route][day,tw_x],np.append(test_data[route][day-7:day,tw_y],Weather_test[day][tw_y,-1])).reshape(1,14)
            test_data[route][day][tw_y] = clfs[route][i].predict(test)

## 输出

In [13]:
out_data = test_data;
## 数据输出
from datetime import datetime
from datetime import timedelta
from pandas import DataFrame


intersection = []
tollgate = []
time_window = []
avg_time = []
routes = ['A2','A3',"B1","B3","C1","C3"]
for route in routes:
    n_day = 7
    start_time = datetime.strptime('2016-10-18 08:00:00',"%Y-%m-%d %H:%M:%S")
    for day in range(n_day):
        starttime = start_time + timedelta(days=day)
        i = 0
        for k in range(24,30): #(51,57)（24，30）
            time_window.append('\"[' + (starttime + timedelta(seconds=1200*i)).strftime("%Y-%m-%d %H:%M:%S")\
                               + "," + (starttime + timedelta(seconds=1200*i+1200)).strftime("%Y-%m-%d %H:%M:%S") + ')\"')
            intersection.append(str(route[0]))
            tollgate.append(str(route[1]))
            avg_time.append(out_data[route][day][k])
            i+=1
            
for route in routes:
    n_day = 7
    start_time = datetime.strptime('2016-10-18 17:00:00',"%Y-%m-%d %H:%M:%S")
    for day in range(n_day):
        starttime = start_time + timedelta(days=day)
        i = 0
        for k in range(51,57): #(51,57)（24，30）
            time_window.append('\"[' + (starttime + timedelta(seconds=1200*i)).strftime("%Y-%m-%d %H:%M:%S")\
                               + "," + (starttime + timedelta(seconds=1200*i+1200)).strftime("%Y-%m-%d %H:%M:%S") + ')\"')
            intersection.append(str(route[0]))
            tollgate.append(str(route[1]))
            avg_time.append(out_data[route][day][k])
            i+=1            
            
d = {"intersection_id":intersection,"tollgate_id":tollgate,"time_window":time_window,"avg_travel_time":avg_time}
pd = DataFrame(data=d)
pd.to_csv('out.csv',index=False,columns=["intersection_id","tollgate_id","time_window","avg_travel_time"])